In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv

### Prepare data
lyrics = pd.read_csv('../input/scrapped-lyrics-from-6-genres/lyrics-data.csv')
lyrics = lyrics[lyrics['language']=='en']

#Only keep popular artists, with genre Rock/Pop and popularity high enough
artists = pd.read_csv('../input/scrapped-lyrics-from-6-genres/artists-data.csv')
#artists = artists[(artists['Genres'].str.\ & (artists['Popularity']>5)]
df = lyrics.merge(artists[['Artist', 'Genres', 'Link']], left_on='ALink', right_on='Link', how='inner')
df = df.drop(columns=['ALink','SLink','language','Link'])
print(df)
df.to_csv('mycsvfile.csv',index=True)

                                                    SName  \
0                                        Careless Whisper   
1       Could You Be Loved / Citação Musical do Rap: S...   
2                                  Cruisin' (Part. Saulo)   
3                                                    Easy   
4                       For Your Babies (The Voice cover)   
...                                                   ...   
191382                                        The Waiting   
191383                              Too Early For The Sky   
191384      Warsaw 1943 (I Never Betrayed The Revolution)   
191385                         When The System Has Fallen   
191386                                Woman Be My Country   

                                                    Lyric         Artist  \
0       I feel so unsure\nAs I take your hand and lead...  Ivete Sangalo   
1       Don't let them fool, ya\nOr even try to school...  Ivete Sangalo   
2       Baby, let's cruise, away from h

In [44]:
type(df['Genres'][0])

str

In [31]:
df2=df

In [32]:
len(df2)

191387

In [55]:
def recommend_song(last_word):
    df1=df.copy()
    df1['Lyric'] = df1['Lyric'].str.split().str[:5]
    #print(df1['Lyric'])
    rec_song=[]
    for i in range(len(df1)):
        k=df1['Lyric'][i]
        #print(k)
        for j in range(len(k)):
            if k[j].lower()==last_word.lower():
        # if last_word in df1[i]['Lyric']:
                rec_song.append([df1['SName'][i],df1['Genres'][i].split('; ')])
    return rec_song
#     print(rec_song)

recommend_song("love")
    

[['More Than Words', ['Pop', 'Axé', 'Romântico']],
 ['Hello', ['Pop', 'R&B', 'Black Music']],
 ['Hold Up', ['Pop', 'R&B', 'Black Music']],
 ['At Last', ['Pop', 'R&B', 'Black Music']],
 ['Be With You', ['Pop', 'R&B', 'Black Music']],
 ['Blow', ['Pop', 'R&B', 'Black Music']],
 ['Blow Remix (Feat. Pharrell Williams)', ['Pop', 'R&B', 'Black Music']],
 ['Crazy Feelings', ['Pop', 'R&B', 'Black Music']],
 ['Dangerously In Love (He Loves Me) (Remix)', ['Pop', 'R&B', 'Black Music']],
 ['Dangerously In Love 2', ['Pop', 'R&B', 'Black Music']],
 ['Love On Top (Dj Escape & Tony Coluccio Remix)',
  ['Pop', 'R&B', 'Black Music']],
 ['Naughty Girl', ['Pop', 'R&B', 'Black Music']],
 ['Naughty Girl', ['Pop', 'R&B', 'Black Music']],
 ['Perfect Duet (With Ed Sheeran)', ['Pop', 'R&B', 'Black Music']],
 ['Still in Love (Kissing You)', ['Pop', 'R&B', 'Black Music']],
 ['Xo', ['Pop', 'R&B', 'Black Music']],
 ['XO (Monsieur Adi Remix)', ['Pop', 'R&B', 'Black Music']],
 ['Hate That I Love You (feat. Ne-yo)', ['

In [66]:
def recommend_song_genre(genre):
    rec_song=recommend_song('love')
    genre=[i.lower() for i in genre]
    rec_song=[[j,[y.lower() for y in i]] for j,i in rec_song]
    ans=[]
    for i in range(len(rec_song)):
        
        lst1=list(set(genre)&set(rec_song[i][1]))
        if len(lst1)>0:
            ans.append([len(lst1),rec_song[i][0]])
    ans.sort(reverse=True)
    lst=[]
    for k,s in ans:
        lst.append(s)
    print(lst)
    
recommend_song_genre(['poP'])
    

['off the table (With The Weeknd)', 'obvious', 'Zombie', 'Yours', 'Your Love V2', 'Your Heart Is My Piñata (I Love You Always Forever)', 'Young Love', "You've Made Me So Very Happy", "You're My Sunday", "You're Gonna Make Me Lonesome When You Go", 'You and Me', 'You Say', 'You Only Love', 'You Never Know', 'You Know That I Want To', 'You Hold My Heart', "You Can't Hurry Love", 'You', 'Yeah Yeah (feat. Travie Mccoy)', 'Xo', 'XO (Monsieur Adi Remix)', 'Wound in the Heart', 'World Family Tree', 'Wonderful', 'Woman In Chains (feat. Oleta Adams)', 'Woman', 'Wings Of Love', 'Wind It', 'Wimmin', 'Wildcat', 'Wild Wild Love (Feat. G.R.L)', 'Wild Is the Wind', 'Wild Is the Wind', 'Wild Is the Wind', 'Why do You Love Me', 'Why Should I Love You?', 'Why Must The Show Go On? (Phil Oakey & Giorgio Moroder)', 'Why Is Love So Hard To Find?', 'Who You Love', 'Who You Love', 'Who Needs Love Like That', 'Who Is Gonna Love Me?', 'Who I Am', 'Who', 'Whispers From A Spiritual Garden', 'Whisper You Love Me B

In [29]:
print(len(df1))

NameError: name 'df1' is not defined